<a href="https://colab.research.google.com/github/space-owner/Tensorflow-2/blob/main/Word2Vec.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import io
import re
import string
import tqdm

import numpy as np

import tensorflow as tf
from tensorflow.keras import layers
assert tf.__version__[0] > "1"

In [ ]:
SEED = 42
AUTOTUNE = tf.data.AUTOTUNE

In [ ]:
sentence = "The wide road shimmered in the hot sun"
tokens = list(sentence.lower().split(" "))
print(">>> tokens length = ", len(tokens))
print(">>> tokens =", tokens)

>>> tokens length =  8
>>> tokens = ['the', 'wide', 'road', 'shimmered', 'in', 'the', 'hot', 'sun']


In [ ]:
vocab, index = {}, 1
vocab['<pad>'] = 0
for token in tokens:
    if token not in vocab:
        vocab[token] = index
        index += 1
vocab_size = len(vocab)
print(">>> vocab =", vocab)

>>> vocab = {'<pad>': 0, 'the': 1, 'wide': 2, 'road': 3, 'shimmered': 4, 'in': 5, 'hot': 6, 'sun': 7}


In [ ]:
inverse_vocab = {index: token for token, index in vocab.items()}
print(">>> inverse_vocab =", inverse_vocab)

>>> inverse_vocab = {0: '<pad>', 1: 'the', 2: 'wide', 3: 'road', 4: 'shimmered', 5: 'in', 6: 'hot', 7: 'sun'}


In [ ]:
example_sequence = [vocab[word] for word in tokens]
print(">>> example sequence =", example_sequence)

>>> example sequence = [1, 2, 3, 4, 5, 1, 6, 7]


In [ ]:
window_size = 2
positive_skip_grams, _ = tf.keras.preprocessing.sequence.skipgrams(
    example_sequence,
    vocabulary_size=vocab_size,
    window_size=window_size,
    negative_samples=0)
print(">>> length of positive skip grams =", len(positive_skip_grams))

>>> length of positive skip grams = 26


In [ ]:
for target, context in positive_skip_grams[:5]:
    print(f"({target}, {context}): ({inverse_vocab[target]}, {inverse_vocab[context]})")

(6, 1): (hot, the)
(3, 5): (road, in)
(4, 2): (shimmered, wide)
(2, 4): (wide, shimmered)
(4, 1): (shimmered, the)


In [ ]:
target_word, context_word = positive_skip_grams[0]

num_ns = 4

context_class = tf.reshape(tf.constant(context_word, dtype="int64"), (1, 1))
negative_sampling_candidates, _, _ = tf.random.log_uniform_candidate_sampler(
    true_classes=context_class,
    num_true=1,
    num_sampled=num_ns,
    unique=True,
    range_max=vocab_size,
    seed=SEED,
    name="negative_sampling"
)
print(">>> negative_sampling_candidates =", negative_sampling_candidates)
print([inverse_vocab[index.numpy()] for index in negative_sampling_candidates])

>>> negative_sampling_candidates = tf.Tensor([2 1 4 3], shape=(4,), dtype=int64)
['wide', 'the', 'shimmered', 'road']


In [ ]:
print(">>> negative_sampling_candidates =", negative_sampling_candidates)

>>> negative_sampling_candidates = tf.Tensor([2 1 4 3], shape=(4,), dtype=int64)


In [ ]:
negative_sampling_candidates = tf.expand_dims(negative_sampling_candidates, 1)

context = tf.concat([context_class, negative_sampling_candidates], 0)

label = tf.constant([1] + [0]*num_ns, dtype="int64")

target = tf.squeeze(target_word)
context = tf.squeeze(context)
label = tf.squeeze(label)

print(f"target_index    : {target}")
print(f"target_word     : {inverse_vocab[target_word]}")
print(f"context_indices : {context}")
print(f"context_words   : {[inverse_vocab[c.numpy()] for c in context]}")
print(f"label           : {label}")

target_index    : 6
target_word     : hot
context_indices : [1 2 1 4 3]
context_words   : ['the', 'wide', 'the', 'shimmered', 'road']
label           : [1 0 0 0 0]


In [ ]:
print("target  :", target)
print("context :", context)
print("label   :", label)

target  : tf.Tensor(6, shape=(), dtype=int32)
context : tf.Tensor([1 2 1 4 3], shape=(5,), dtype=int64)
label   : tf.Tensor([1 0 0 0 0], shape=(5,), dtype=int64)


In [ ]:
sampling_table = tf.keras.preprocessing.sequence.make_sampling_table(size=10)
print(">> sampling_table =", sampling_table)

>> sampling_table = [0.00315225 0.00315225 0.00547597 0.00741556 0.00912817 0.01068435
 0.01212381 0.01347162 0.01474487 0.0159558 ]


In [ ]:
def generate_training_data(sequences, window_size, num_ns, vocab_size, seed):
    targets, contexts, labels = [], [], []

    sampling_table = tf.keras.preprocessing.sequence.make_sampling_table(vocab_size)

    for sequence in tqdm.tqdm(sequences):
        positive_skip_grams, _ = tf.keras.preprocessing.sequence.skipgrams(
            sequence,
            vocabulary_size=vocab_size,
            sampling_table=sampling_table,
            window_size=window_size,
            negative_samples=0)

        for target_word, context_word in positive_skip_grams:
            context_class = tf.expand_dims(
                tf.constant([context_word], dtype="int64"), 1)
            negative_sampling_candidates, _, _ = tf.random.log_uniform_candidate_sampler(
                true_classes=context_class,
                num_true=1,
                num_sampled=num_ns,
                unique=True,
                range_max=vocab_size,
                seed=SEED,
                name="negative_sampling")

            negative_sampling_candidates = tf.expand_dims(
                negative_sampling_candidates, 1)

            context = tf.concat([context_class, negative_sampling_candidates], 0)
            label = tf.constant([1] + [0]*num_ns, dtype="int64")

            targets.append(target_word)
            contexts.append(context)
            labels.append(label)
    return targets, contexts, labels

In [ ]:
path_to_file = tf.keras.utils.get_file(
    'shakespeare.txt', 'https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt')

1130496/1115394 [==============================] - 0s 0us/step


In [ ]:
with open(path_to_file) as f: 
    lines = f.read().splitlines()
for line in lines[:20]:
    print(">>> line =", line)

>>> line = First Citizen:
>>> line = Before we proceed any further, hear me speak.
>>> line = 
>>> line = All:
>>> line = Speak, speak.
>>> line = 
>>> line = First Citizen:
>>> line = You are all resolved rather to die than to famish?
>>> line = 
>>> line = All:
>>> line = Resolved. resolved.
>>> line = 
>>> line = First Citizen:
>>> line = First, you know Caius Marcius is chief enemy to the people.
>>> line = 
>>> line = All:
>>> line = We know't, we know't.
>>> line = 
>>> line = First Citizen:
>>> line = Let us kill him, and we'll have corn at our own price.


In [ ]:
text_ds = tf.data.TextLineDataset(path_to_file).filter(lambda x: tf.cast(tf.strings.length(x), bool))

In [ ]:
def custom_standardization(input_data):
    lowercase = tf.strings.lower(input_data)
    return tf.strings.regex_replace(lowercase, '[%s]' % re.escape(string.punctuation), '')

vocab_size = 4096
sequence_length = 10

vectorize_layer = layers.TextVectorization(
    standardize=custom_standardization,
    max_tokens=vocab_size,
    output_mode='int',
    output_sequence_length=sequence_length)

In [ ]:
vectorize_layer.adapt(text_ds.batch(1024))

In [ ]:
inverse_vocab = vectorize_layer.get_vocabulary()
print(">>> inverse_vocab =", inverse_vocab[:20])

>>> inverse_vocab = ['', '[UNK]', 'the', 'and', 'to', 'i', 'of', 'you', 'my', 'a', 'that', 'in', 'is', 'not', 'for', 'with', 'me', 'it', 'be', 'your']


In [ ]:
text_vector_ds = text_ds.batch(1024).prefetch(AUTOTUNE).map(vectorize_layer).unbatch()

In [ ]:
sequences = list(text_vector_ds.as_numpy_iterator())
print(">>> lenght of sequences ", len(sequences))

>>> lenght of sequences  32777


In [ ]:
for seq in sequences[:5]:
    print(f"{seq} => {[inverse_vocab[i] for i in seq]}")

[ 89 270   0   0   0   0   0   0   0   0] => ['first', 'citizen', '', '', '', '', '', '', '', '']
[138  36 982 144 673 125  16 106   0   0] => ['before', 'we', 'proceed', 'any', 'further', 'hear', 'me', 'speak', '', '']
[34  0  0  0  0  0  0  0  0  0] => ['all', '', '', '', '', '', '', '', '', '']
[106 106   0   0   0   0   0   0   0   0] => ['speak', 'speak', '', '', '', '', '', '', '', '']
[ 89 270   0   0   0   0   0   0   0   0] => ['first', 'citizen', '', '', '', '', '', '', '', '']


In [ ]:
targets, contexts, labels = generate_training_data(
    sequences=sequences,
    window_size=2,
    num_ns=4,
    vocab_size=vocab_size,
    seed=SEED)

targets = np.array(targets)
contexts = np.array(contexts)[:,:,0]
labels = np.array(labels)

print(f"targets.shape: {targets.shape}")
print(f"contexts.shape: {contexts.shape}")
print(f"labels.shape: {labels.shape}")

100%|██████████| 32777/32777 [00:13<00:00, 2384.95it/s]


targets.shape: (65204,)
contexts.shape: (65204, 5)
labels.shape: (65204, 5)


In [ ]:
BATCH_SIZE = 1024
BUFFER_SIZE = 10000
dataset = tf.data.Dataset.from_tensor_slices(((targets, contexts), labels))
dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)
print(">>> dataset =", dataset)

>>> dataset = <BatchDataset shapes: (((1024,), (1024, 5)), (1024, 5)), types: ((tf.int64, tf.int64), tf.int64)>


In [30]:
dataset = dataset.cache().prefetch(buffer_size=AUTOTUNE)
print(">>> dataset =", dataset)

>>> dataset = <PrefetchDataset shapes: (((1024,), (1024, 5)), (1024, 5)), types: ((tf.int64, tf.int64), tf.int64)>
